In [ ]:
import numpy as np
import time
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader
from dgl import model_zoo,DGLGraph
import math
import networkx as nx
import matplotlib.pyplot as plt
from utils_merge import Meter, set_random_seed, collate, EarlyStopping,load_brl_dataset
import argparse
# from sklearn import svm, datasets
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import plot_confusion_matrix
import seaborn as sn
import pandas as pd
from sklearn import metrics
import collections
import sys; sys.argv=['']; del sys
import torch.nn as nn
import torch.nn.functional as F
from dgl.contrib.deprecation import deprecated
from dgl.nn.pytorch import Set2Set, NNConv
    
class ImputationNet(nn.Module):
    def __init__(self):
        super(ImputationNet,self).__init__()
        
        self.fc1 = nn.Linear(6*7+7, 512)
        self.fc2 = nn.Linear(512,512)
        self.fc3 = nn.Linear(512,512)
        self.fc_last = nn.Linear(512,359*7)
        self.tanh=nn.Tanh()
        
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.tanh(self.fc_last(x))#self.fc_last(x)#
        return x

class MPNNModel(nn.Module):
    @deprecated('Import MPNNPredictor from dgllife.model instead.', 'class')
    def __init__(self,
                 node_input_dim=15,
                 edge_input_dim=5,
                 #output_dim=12,
                 node_hidden_dim=64,
                 edge_hidden_dim=128,#64,#
                 num_step_message_passing=6,#4,#
                 num_step_set2set=6,#4,#
                 num_layer_set2set=3):#2):#
        super(MPNNModel, self).__init__()

        self.num_step_message_passing = num_step_message_passing
        self.lin0 = nn.Linear(node_input_dim, node_hidden_dim)
        edge_network = nn.Sequential(
            nn.Linear(edge_input_dim, edge_hidden_dim), nn.ReLU(),
            nn.Linear(edge_hidden_dim, node_hidden_dim * node_hidden_dim))
        self.conv = NNConv(in_feats=node_hidden_dim,
                           out_feats=node_hidden_dim,
                           edge_func=edge_network,
                           aggregator_type='sum')
        self.gru = nn.GRU(node_hidden_dim, node_hidden_dim)

        self.set2set = Set2Set(node_hidden_dim, num_step_set2set, num_layer_set2set)
        self.lin1 = nn.Linear(2 * node_hidden_dim, node_hidden_dim)
        
        self.lin2_1 = nn.Linear(node_hidden_dim, 121)
        self.lin2_2 = nn.Linear(node_hidden_dim, 1)

    def forward(self, g, n_feat, e_feat):
        out = F.relu(self.lin0(n_feat))                 # (B1, H1)
        h = out.unsqueeze(0)                            # (1, B1, H1)
        
        for i in range(self.num_step_message_passing):
            m = F.relu(self.conv(g, out, e_feat))       # (B1, H1)
            out, h = self.gru(m.unsqueeze(0), h)
            out = out.squeeze(0)
            
        out = self.set2set(g, out)
        out = F.relu(self.lin1(out))
        
        out1=self.lin2_1(out)#for classification
        out2=self.lin2_2(out)#for regression
        return [out1,out2]

class ImPrNet(nn.Module):
    def __init__(self, model_disp, model_multitask):
        super(ImPrNet, self).__init__()
        self.model_imputation = model_disp
        self.model_multitask = model_multitask
        
    def forward(self, g,  x, e_feat):#x(n,6*7+7)-disp6,nf
        disp_359 = self.model_imputation(x)#(n,359*7)-disp359
        disp_359=disp_359.view(-1,359,7)
        x=x[:,:6*7].view(-1,6,7)
        
        x=torch.cat((disp_359,x),axis=1)
        x=x.view(len(x)*365,7)
        #print('forwad',len(g),x.shape)
        
        #h = bg.ndata.pop('n_feat')
        #e = bg.edata.pop('e_feat')
        #h, e = h.to(args['device']), e.to(args['device'])
        #for i in range(len(g)):g[i].ndata['n_feat'] =x[i]
        g.ndata['n_feat'] =x
        
        output = self.model_multitask(g,  x, e_feat)
        
        return output
    


print(torch.cuda.current_device())
# torch.cuda.set_device(1)
# print(torch.cuda.current_device())


parser = argparse.ArgumentParser(description='Molecule Regression')
parser.add_argument('-m', '--model', type=str,default='MPNN',help='Model to use')#choices=['MPNN', 'SCHNET', 'MGCN', 'AttentiveFP'],
#parser.add_argument('-d', '--dataset', type=str, default='bridge',help='Dataset to use')#choices=['Alchemy', 'Aromaticity'],                
parser.add_argument('-p', '--pre-trained', action='store_true', default=False, help='Whether to skip training and use a pre-trained model')
args = parser.parse_args().__dict__
training_setting= {
    'random_seed': 0,
    'batch_size': 64,#64,#
    'num_epochs': 2000,#900,
    'node_in_feats': 7,
    'edge_in_feats': 6,
    #'output_dim': 120,
    'lr': 0.00001,#0.001,#
    'patience': 300,#20,
    'metric_name': 'l1',#'roc_auc',#
    'weight_decay': 0,
    'n_task':41,
}
args.update(training_setting)
args['device'] = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
args['data_path_x']='data/data_mlp/single/input6/'#'data/data_mpnn/'
args['data_path_label']='data/label_multitask/'
set_random_seed(args['random_seed'])


train_loader,val_loader,test_loader=load_brl_dataset(args)#for batch_id, batch_data in enumerate(train_loader):bg, labels = batch_data;print(a)

#model = load_model(args)
model_i = ImputationNet()
model_p = MPNNModel(node_input_dim=args['node_in_feats'],
                  edge_input_dim=args['edge_in_feats'])

# Load state dicts
model_i.load_state_dict(torch.load('model_saved/mlp/single/imputation/last.pth')['model_state_dict'])
model_p.load_state_dict(torch.load('model_saved/multitask/model2_5000/early_stop.pth')['model_state_dict'])

model = ImPrNet(model_i, model_p)
model.to(args['device'])

loss_fn_cablenumber =nn.CrossEntropyLoss()
loss_fn_area =nn.L1Loss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])
stopper = EarlyStopping(mode='lower', patience=args['patience'],
                        filename='model_saved/finetuning/early_stop.pth')


def run_a_train_epoch_cr(args, epoch, model, data_loader,
                      loss_criterion_cablenumber,loss_criterion_area, optimizer):
    model.train()
    train_meter = Meter()
    correct=0
    total=0
    for batch_id, batch_data in enumerate(data_loader):
        bg, h, labels_cablenumber,labels_area = batch_data
        labels_cablenumber = labels_cablenumber.to(args['device'])
        labels_area = labels_area.to(args['device'])
        labels=[labels_cablenumber,labels_area]
        #prediction = regress(args, model, bg)
        #h = bg.ndata.pop('n_feat')
        e = bg.edata.pop('e_feat')
        h, e = h.to(args['device']), e.to(args['device'])
        prediction = model(bg, h, e)
        
        loss_cablenumber = (loss_criterion_cablenumber(prediction[0], labels_cablenumber).float()).mean()
        loss_area = (loss_criterion_area(prediction[1],labels_area).float()).mean()
        loss=(loss_cablenumber+loss_area)
       
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_meter.update(prediction[1], labels_area)
        
        _, predicted = torch.max(prediction[0].data, 1)
        total += labels_cablenumber.size(0)
        correct += (predicted == labels_cablenumber).sum().item();del predicted
        
    total_score_acc=100-100 * correct / total
    total_score = np.mean(train_meter.compute_metric(args['metric_name']))
    print('epoch {:d}/{:d}, training {} {:.4f} / accuracy(%) {:.4f}'.format(
        epoch + 1, args['num_epochs'], args['metric_name'], total_score, total_score_acc)) 
    #print('epoch {:d}/{:d}, training total_score_acc {:.4f}'.format(epoch + 1, args['num_epochs'], total_score_acc)) 
    
def run_an_eval_epoch_cr(args, model, data_loader):
    model.eval()
    eval_meter = Meter()
    correct=0
    total=0
    with torch.no_grad():
        for batch_id, batch_data in enumerate(data_loader):
            bg,h, labels_cablenumber,labels_area = batch_data
            labels_cablenumber = labels_cablenumber.to(args['device'])
            labels_area = labels_area.to(args['device'])
            labels=[labels_cablenumber,labels_area]
            #prediction = regress(args, model, bg)
            #h = bg.ndata.pop('n_feat')
            e = bg.edata.pop('e_feat')
            h, e = h.to(args['device']), e.to(args['device'])
            prediction = model(bg, h, e)
            
            
            eval_meter.update(prediction[1], labels_area)
            
            _, predicted = torch.max(prediction[0].data, 1)
            total += labels_cablenumber.size(0)
            correct += (predicted == labels_cablenumber).sum().item();del predicted
        total_score_acc=100-100 * correct / total
        total_score = np.mean(eval_meter.compute_metric(args['metric_name']))
    return total_score,total_score_acc


In [ ]:
# torch.save({'model_state_dict': model.state_dict(),
#             'optimizer': optimizer.state_dict()}, 'model_saved/multitask/model2_5000/1.pth')
state=torch.load('model_saved/finetuning/early_stop.pth') 
model.load_state_dict(state['model_state_dict'])
optimizer.load_state_dict(state['optimizer'])
for param_group in optimizer.param_groups: print(param_group['lr'])
# for param_group in optimizer.param_groups: param_group['lr'] = param_group['lr']*0.1
# for param_group in optimizer.param_groups: print(param_group['lr'])
 

# training set

In [76]:
model.eval()
#eval_meter = Meter()
cn_p=[];area_p=[];cn_t=[];area_t=[]
with torch.no_grad():
    for batch_id, batch_data in enumerate(train_loader):
        bg,h, labels_cablenumber,labels_area = batch_data
        
        labels_cablenumber = labels_cablenumber.to(args['device'])
        labels_area = labels_area.to(args['device'])
        cn_t.append(labels_cablenumber.cpu().detach().numpy())
        area_t.append(labels_area.cpu().detach().numpy())
        
        e = bg.edata.pop('e_feat')
        h, e = h.to(args['device']), e.to(args['device'])
        prediction = model(bg, h, e)
        
        
        cn_p.append(prediction[0].cpu().detach().numpy())
        area_p.append(prediction[1].cpu().detach().numpy())

cn_p=np.concatenate(cn_p,axis=0)
cn_p=np.argmax(cn_p,axis=1)
area_p=np.concatenate(area_p,axis=0)

cn_t=np.concatenate(cn_t,axis=0)
area_t=np.concatenate(area_t,axis=0)

print(cn_p.shape,area_p.shape,cn_t.shape,area_t.shape)

acc=np.sum(cn_p==cn_t)/len(cn_p)
print('acc: ',acc)
#plt.plot(area_t,area_p,'.')

w_idx=np.where(cn_p!=cn_t)
c_idx=np.where(cn_p==cn_t)

area=np.load('data/label_multitask/a_label_train.npz')['arr_0']
print('acc mean',np.mean(area[w_idx]),np.mean(area[c_idx]))

(7276,) (7276, 1) (7276,) (7276, 1)
acc:  0.9677020340846619
acc mean 0.4802544765957446 0.49932442536571514


In [ ]:
import sklearn.metrics as metrics
t_area=area_t.copy();p_area=area_p.copy()
#plt.plot(p_area.flatten(),t_area.flatten(),'.',color='darkblue',alpha=0.2)
plt.plot(t_area[c_idx].flatten(),p_area[c_idx].flatten(),'.',color='deepskyblue',alpha=0.3)
plt.plot(t_area[w_idx].flatten(),p_area[w_idx].flatten(),'.',color='crimson',alpha=0.3)
#plt.ylim(np.min([a_p,a_t]), np.max([a_p,a_t]))
#plt.xlim(np.min([a_p,a_t]), np.max([a_p,a_t]))
plt.ylim(-0.1,1.1)
plt.xlim(-0.1,1.1)
plt.xlabel('actual')
plt.ylabel('pred')
plt.grid()
#plt.savefig('images/train_area.png', bbox_inches = 'tight')
plt.show()

y_t=area_t.copy();y_p=area_p.copy()
print(y_t.shape,y_p.shape)#(
import sklearn.metrics as metrics
mae = metrics.mean_absolute_error(y_t, y_p)
mse = metrics.mean_squared_error(y_t, y_p)
rmse=np.sqrt(mse)
print('mae', mae, '| rmse:',rmse)
print('actual mean', np.mean(y_t),'| pred mean',np.mean(y_p))
print(np.corrcoef(y_t.flatten(),y_p.flatten()))
print('rmse/range',rmse/(np.max(y_t)-np.min(y_t)))
print('mape',np.mean(np.abs(y_t-y_p)/y_t))
iqr= np.subtract(*np.percentile(y_t, [75, 25]))
print('rmse/iqr',rmse/iqr)
print('rmse/mean',rmse/np.mean(y_t))
print('actual min max', np.min(y_t),np.max(y_t))
print('pred min max', np.min(y_p),np.max(y_p))

In [ ]:
t_area=area_t.copy();p_area=area_p.copy()
#plt.plot(p_area.flatten(),t_area.flatten(),'.',color='darkblue',alpha=0.2)
plt.plot(t_area[c_idx].flatten(),p_area[c_idx].flatten(),'.',color='deepskyblue',alpha=0.5)
##plt.plot(t_area[w_idx].flatten(),p_area[w_idx].flatten(),'.',color='red',alpha=0.3)
#plt.ylim(np.min([a_p,a_t]), np.max([a_p,a_t]))
#plt.xlim(np.min([a_p,a_t]), np.max([a_p,a_t]))
plt.ylim(-0.1,1.1)
plt.xlim(-0.1,1.1)
plt.xlabel('actual')
plt.ylabel('pred')
plt.grid()
##plt.savefig('images/test_area_c.png', bbox_inches = 'tight')
plt.show()

y_t=area_t[c_idx].copy();y_p=area_p[c_idx].copy()
print(y_t.shape,y_p.shape)#(
import sklearn.metrics as metrics
mae = metrics.mean_absolute_error(y_t, y_p)
mse = metrics.mean_squared_error(y_t, y_p)
rmse=np.sqrt(mse)
print('mae', mae, '| rmse:',rmse)
print('actual mean', np.mean(y_t),'| pred mean',np.mean(y_p))
print(np.corrcoef(y_t.flatten(),y_p.flatten()))
print('rmse/range',rmse/(np.max(y_t)-np.min(y_t)))
print('mape',np.mean(np.abs(y_t-y_p)/y_t))
iqr= np.subtract(*np.percentile(y_t, [75, 25]))
print('rmse/iqr',rmse/iqr)
print('rmse/mean',rmse/np.mean(y_t))
print('actual min max', np.min(y_t),np.max(y_t))
print('pred min max', np.min(y_p),np.max(y_p))

In [ ]:
import sklearn.metrics as metrics
t_area=area_t.copy();p_area=area_p.copy()
#plt.plot(p_area.flatten(),t_area.flatten(),'.',color='darkblue',alpha=0.2)
##plt.plot(t_area[c_idx].flatten(),p_area[c_idx].flatten(),'.',color='aqua',alpha=0.3)
plt.plot(t_area[w_idx].flatten(),p_area[w_idx].flatten(),'.',color='crimson',alpha=0.3)
#plt.ylim(np.min([a_p,a_t]), np.max([a_p,a_t]))
#plt.xlim(np.min([a_p,a_t]), np.max([a_p,a_t]))
plt.ylim(-0.1,1.1)
plt.xlim(-0.1,1.1)
plt.xlabel('actual')
plt.ylabel('pred')
plt.grid()
##plt.savefig('images/test_area_w.png', bbox_inches = 'tight')
plt.show()

y_t=area_t[w_idx].copy();y_p=area_p[w_idx].copy()
print(y_t.shape,y_p.shape)#(
import sklearn.metrics as metrics
mae = metrics.mean_absolute_error(y_t, y_p)
mse = metrics.mean_squared_error(y_t, y_p)
rmse=np.sqrt(mse)
print('mae', mae, '| rmse:',rmse)
print('actual mean', np.mean(y_t),'| pred mean',np.mean(y_p))
print(np.corrcoef(y_t.flatten(),y_p.flatten()))
print('rmse/range',rmse/(np.max(y_t)-np.min(y_t)))
print('mape',np.mean(np.abs(y_t-y_p)/y_t))
iqr= np.subtract(*np.percentile(y_t, [75, 25]))
print('rmse/iqr',rmse/iqr)
print('rmse/mean',rmse/np.mean(y_t))
print('actual min max', np.min(y_t),np.max(y_t))
print('pred min max', np.min(y_p),np.max(y_p))

In [ ]:
print('min, max',np.min(area_t),np.max(area_t))
break_case=1e-06
damaged_list=np.arange(0.01,0.99,0.1)
stable_case=0.9999

area_idx=np.where(area_t==break_case)[0]
acc_case=np.sum(cn_p[area_idx]==cn_t[area_idx])/len(cn_p[area_idx])
print('break_case',acc_case)
for i in damaged_list:
    low_bound=round(i,3)
    upper_bound=round(i+0.09,3)
    if upper_bound==1: upper_bound=0.99
    area_idx=np.where((area_t>=low_bound) & (area_t<=upper_bound))[0]
    acc_case=np.sum(cn_p[area_idx]==cn_t[area_idx])/len(cn_p[area_idx])
    print(low_bound,'~',upper_bound,':',acc_case)
area_idx=np.where(area_t>=stable_case)[0]
acc_case=np.sum(cn_p[area_idx]==cn_t[area_idx])/len(cn_p[area_idx])
print('stable_case',acc_case)

# test set

In [ ]:
model.eval()
#eval_meter = Meter()
cn_p=[];area_p=[];cn_t=[];area_t=[]
with torch.no_grad():
    for batch_id, batch_data in enumerate(test_loader):
        bg,h, labels_cablenumber,labels_area = batch_data
        
        labels_cablenumber = labels_cablenumber.to(args['device'])
        labels_area = labels_area.to(args['device'])
        cn_t.append(labels_cablenumber.cpu().detach().numpy())
        area_t.append(labels_area.cpu().detach().numpy())
        
        e = bg.edata.pop('e_feat')
        h, e = h.to(args['device']), e.to(args['device'])
        prediction = model(bg, h, e)
        
        
        cn_p.append(prediction[0].cpu().detach().numpy())
        area_p.append(prediction[1].cpu().detach().numpy())

cn_p=np.concatenate(cn_p,axis=0)
cn_p=np.argmax(cn_p,axis=1)
area_p=np.concatenate(area_p,axis=0)

cn_t=np.concatenate(cn_t,axis=0)
area_t=np.concatenate(area_t,axis=0)

print(cn_p.shape,area_p.shape,cn_t.shape,area_t.shape)

acc=np.sum(cn_p==cn_t)/len(cn_p)
print('acc: ',acc)
#plt.plot(area_t,area_p,'.')

w_idx=np.where(cn_p!=cn_t)
c_idx=np.where(cn_p==cn_t)

area=np.load('data/label_multitask/a_label_test.npz')['arr_0']
print('acc mean',np.mean(area[w_idx]),np.mean(area[c_idx]))

In [ ]:
import sklearn.metrics as metrics
t_area=area_t.copy();p_area=area_p.copy()
#plt.plot(p_area.flatten(),t_area.flatten(),'.',color='darkblue',alpha=0.2)
plt.plot(t_area[c_idx].flatten(),p_area[c_idx].flatten(),'.',color='deepskyblue',alpha=0.5)
plt.plot(t_area[w_idx].flatten(),p_area[w_idx].flatten(),'.',color='crimson',alpha=0.3)
#plt.ylim(np.min([a_p,a_t]), np.max([a_p,a_t]))
#plt.xlim(np.min([a_p,a_t]), np.max([a_p,a_t]))
plt.ylim(-0.1,1.1)
plt.xlim(-0.1,1.1)
plt.xlabel('actual')
plt.ylabel('pred')
plt.grid()
plt.savefig('images/test_area.png', bbox_inches = 'tight')
plt.show()

y_t=area_t.copy();y_p=area_p.copy()
print(y_t.shape,y_p.shape)#(
import sklearn.metrics as metrics
mae = metrics.mean_absolute_error(y_t, y_p)
mse = metrics.mean_squared_error(y_t, y_p)
rmse=np.sqrt(mse)
print('mae', mae, '| rmse:',rmse)
print('actual mean', np.mean(y_t),'| pred mean',np.mean(y_p))
print(np.corrcoef(y_t.flatten(),y_p.flatten()))
print('rmse/range',rmse/(np.max(y_t)-np.min(y_t)))
print('mape',np.mean(np.abs(y_t-y_p)/y_t))
iqr= np.subtract(*np.percentile(y_t, [75, 25]))
print('rmse/iqr',rmse/iqr)
print('rmse/mean',rmse/np.mean(y_t))
print('actual min max', np.min(y_t),np.max(y_t))
print('pred min max', np.min(y_p),np.max(y_p))

In [ ]:
t_area=area_t.copy();p_area=area_p.copy()
#plt.plot(p_area.flatten(),t_area.flatten(),'.',color='darkblue',alpha=0.2)
plt.plot(t_area[c_idx].flatten(),p_area[c_idx].flatten(),'.',color='deepskyblue',alpha=0.5)
##plt.plot(t_area[w_idx].flatten(),p_area[w_idx].flatten(),'.',color='red',alpha=0.3)
#plt.ylim(np.min([a_p,a_t]), np.max([a_p,a_t]))
#plt.xlim(np.min([a_p,a_t]), np.max([a_p,a_t]))
plt.ylim(-0.1,1.1)
plt.xlim(-0.1,1.1)
plt.xlabel('actual')
plt.ylabel('pred')
plt.grid()
plt.savefig('images/test_area_c.png', bbox_inches = 'tight')
plt.show()

y_t=area_t[c_idx].copy();y_p=area_p[c_idx].copy()
print(y_t.shape,y_p.shape)#(
import sklearn.metrics as metrics
mae = metrics.mean_absolute_error(y_t, y_p)
mse = metrics.mean_squared_error(y_t, y_p)
rmse=np.sqrt(mse)
print('mae', mae, '| rmse:',rmse)
print('actual mean', np.mean(y_t),'| pred mean',np.mean(y_p))
print(np.corrcoef(y_t.flatten(),y_p.flatten()))
print('rmse/range',rmse/(np.max(y_t)-np.min(y_t)))
print('mape',np.mean(np.abs(y_t-y_p)/y_t))
iqr= np.subtract(*np.percentile(y_t, [75, 25]))
print('rmse/iqr',rmse/iqr)
print('rmse/mean',rmse/np.mean(y_t))
print('actual min max', np.min(y_t),np.max(y_t))
print('pred min max', np.min(y_p),np.max(y_p))

In [ ]:
import sklearn.metrics as metrics
t_area=area_t.copy();p_area=area_p.copy()
#plt.plot(p_area.flatten(),t_area.flatten(),'.',color='darkblue',alpha=0.2)
##plt.plot(t_area[c_idx].flatten(),p_area[c_idx].flatten(),'.',color='aqua',alpha=0.3)
plt.plot(t_area[w_idx].flatten(),p_area[w_idx].flatten(),'.',color='crimson',alpha=0.3)
#plt.ylim(np.min([a_p,a_t]), np.max([a_p,a_t]))
#plt.xlim(np.min([a_p,a_t]), np.max([a_p,a_t]))
plt.ylim(-0.1,1.1)
plt.xlim(-0.1,1.1)
plt.xlabel('actual')
plt.ylabel('pred')
plt.grid()
plt.savefig('images/test_area_w.png', bbox_inches = 'tight')
plt.show()

y_t=area_t[w_idx].copy();y_p=area_p[w_idx].copy()
print(y_t.shape,y_p.shape)#(
import sklearn.metrics as metrics
mae = metrics.mean_absolute_error(y_t, y_p)
mse = metrics.mean_squared_error(y_t, y_p)
rmse=np.sqrt(mse)
print('mae', mae, '| rmse:',rmse)
print('actual mean', np.mean(y_t),'| pred mean',np.mean(y_p))
print(np.corrcoef(y_t.flatten(),y_p.flatten()))
print('rmse/range',rmse/(np.max(y_t)-np.min(y_t)))
print('mape',np.mean(np.abs(y_t-y_p)/y_t))
iqr= np.subtract(*np.percentile(y_t, [75, 25]))
print('rmse/iqr',rmse/iqr)
print('rmse/mean',rmse/np.mean(y_t))
print('actual min max', np.min(y_t),np.max(y_t))
print('pred min max', np.min(y_p),np.max(y_p))

In [ ]:
print(np.min(area_t),np.max(area_t))
area_list=[1e-06]+list(np.round(np.linspace(0.01,0.99,99),2))+[0.9999]#np.min(area_t)
acc_list=[]
for i in area_list:
    area_idx=np.where(area_t==i)[0]
    acc_list.append(np.sum(cn_p[area_idx]==cn_t[area_idx])/len(cn_p[area_idx]))
print(acc_list)

plt.figure(figsize=(6,3))
plt.plot(area_list,acc_list,'.-',c='darkblue')
plt.grid()
plt.xlabel('area')
plt.ylabel('accuracy')
plt.savefig('images/test_acc_list.png', bbox_inches = 'tight')
plt.show()

In [ ]:
area_idx=np.where(area_t<0.95)[0]
np.sum(cn_p[area_idx]==cn_t[area_idx])/len(cn_p[area_idx])

In [ ]:
area_idx=np.where(area_t<0.90)[0]
np.sum(cn_p[area_idx]==cn_t[area_idx])/len(cn_p[area_idx])

In [ ]:
list(np.arange(0.01,0.99,0.05))

In [ ]:
damaged_list

In [ ]:
print('min, max',np.min(area_t),np.max(area_t))
break_case=1e-06
damaged_list=np.arange(0.01,0.99,0.05)
stable_case=0.9999#exclude 100% -> it dosen't included in test set

area_idx=np.where(area_t==break_case)[0]
acc_case=np.sum(cn_p[area_idx]==cn_t[area_idx])/len(cn_p[area_idx])
print('break_case',acc_case)
for i in damaged_list:
    low_bound=round(i,3)
    upper_bound=round(i+0.04,3)
    if upper_bound==1: upper_bound=0.99
    area_idx=np.where((area_t>=low_bound) & (area_t<=upper_bound))[0]
    acc_case=np.sum(cn_p[area_idx]==cn_t[area_idx])/len(cn_p[area_idx])
    print(low_bound,'~',upper_bound,':',acc_case)
area_idx=np.where(area_t==stable_case)[0]
acc_case=np.sum(cn_p[area_idx]==cn_t[area_idx])/len(cn_p[area_idx])
print('stable_case',acc_case)

In [ ]:
np.arange(0.01,0.99,0.1)

In [ ]:
print('min, max',np.min(area_t),np.max(area_t))
break_case=1e-06
damaged_list=np.arange(0.01,0.99,0.1)
stable_case=0.9999#exclude 100% -> it dosen't included in test set

area_idx=np.where(area_t==break_case)[0]
acc_case=np.sum(cn_p[area_idx]==cn_t[area_idx])/len(cn_p[area_idx])
print('break_case',acc_case)
for i in damaged_list:
    low_bound=round(i,3)
    upper_bound=round(i+0.09,3)
    if upper_bound==1: upper_bound=0.99
    area_idx=np.where((area_t>=low_bound) & (area_t<=upper_bound))[0]
    acc_case=np.sum(cn_p[area_idx]==cn_t[area_idx])/len(cn_p[area_idx])
    print(low_bound,'~',upper_bound,':',acc_case)
area_idx=np.where(area_t==stable_case)[0]
acc_case=np.sum(cn_p[area_idx]==cn_t[area_idx])/len(cn_p[area_idx])
print('stable_case',acc_case)

In [ ]:
print(sum(cn_t==0),sum(cn_p==0))
print(np.where(cn_t==0))
print(np.where(cn_p==0))

idx_ns=np.setdiff1d(np.where(cn_p==0)[0],np.where(cn_t==0)[0])
print(idx_ns)

In [ ]:
area_t[idx_ns]

In [ ]:
print(sum(cn_t!=0),sum(cn_p!=0))
print(np.where(cn_t!=0))
print(np.where(cn_p!=0))

idx_ns=np.intersect1d(np.where(cn_p!=0)[0],np.where(cn_t!=0)[0])
print(len(idx_ns))
area_t[idx_ns]

In [ ]:
len(np.where(area_t[idx_ns]>=0.96)[0])

In [ ]:
len(np.where(area_t[idx_ns]>=0.95)[0])

In [ ]:
np.where(area_t[idx_ns]>0.95)[0]

In [ ]:
area_t[idx_ns][area_t[idx_ns]>0.95]

In [ ]:
cm=metrics.confusion_matrix(cn_t,cn_p)
df_cm = pd.DataFrame(cm, index = [i for i in range(121)],
                  columns = [i for i in range(121)])
plt.figure(figsize = (20,15))
sn.heatmap(df_cm, cmap="Blues")#, annot=True)
plt.ylabel('Actual', fontsize = 15)
plt.xlabel('Predicted', fontsize = 15)
plt.savefig('images/confusion_test.png', bbox_inches = 'tight')
plt.show()

In [ ]:
cn_list=list(range(1,121))
mse_list=[]
for i in cn_list:
    cn_idx=np.where(cn_t==i)[0]
    mse_list.append(metrics.mean_squared_error(area_t[cn_idx], area_p[cn_idx]))
print(mse_list)

plt.figure(figsize=(8,1.5))
plt.bar(cn_list,mse_list,color='darkblue')
plt.xlabel('cable')
plt.ylabel('MSE')
plt.savefig('images/mse_list_test.png', bbox_inches = 'tight')
plt.grid()

In [ ]:
np.where(np.array(mse_list)>0.03)[0]+1

In [ ]:
np.where(np.array(mse_list)==np.min(mse_list))[0]+1